# Up and Running with Tensorflow 

Tensorflow is a powerful open source software library for numerical computation, particularly well suited and fine-tuned for large-scale machine learning.

Basically, we definea graph of computations to perform, and Tensorflow takes that graph and runs it efficiently using optimized C++ code.

Most importantly, Tensorflow can perform parallel operations across multiple CPUs or GPUs, meaning that it can train on huge neural networks with masssive amounts of data. It is mainly behind Google Speech, Google Photos, and Google Search.

## Installation 

In [2]:
import tensorflow as tf

print(tf.__version__)

1.11.0


## Creating a graph and running it 

In [3]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

The code above did not run. In fact, the variables are not even defined. We need to open a Tensorflow session to initialize the variables and compute the function. A *session* is responsible of placing the operations onto the CPU or the GPU. Usually, we close the session to free up resources once the computation is done.

In [6]:
#Open a session
sess = tf.Session()
print('TF session opened')

#Initialize variables
sess.run(x.initializer)
sess.run(y.initializer)

#Compute function
result = sess.run(f)
print(result)

#Close session
sess.close()
print('TF session closed')

TF session opened
42
TF session closed


It is cumbersome to open and close the session manually. Instead, we can use the following syntax to automatically close a TF session:

In [7]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


Also, instead of initializing each variable, we can create a node that will then initialize all variables when it is run:

In [8]:
#Initialize an init node
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


We can also use an `InteractiveSession()` to avoid the `with` block, but we will need to close the session manually.

In [13]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


C:\Users\marco\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1662: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [14]:
sess.close()

Typically, a TS program is split into 2 parts: a construction phase and a an execution phase. During the construction phase, we simply build the computation graph. This represents all equations and steps to train a model Then, the execution phase usually consists of a loop that will repeatedly train on batches of data, gradually improving the model's parameters 

## Managing graphs 